## VT Relaxation with Dissociation in Air

In [45]:
%load_ext autoreload
%autoreload 2

import jax
import jax.numpy as jnp

from compressible_1d import (
    equation_manager_types,
    equation_manager_utils,
    chemistry_types,
    chemistry_utils,
    numerics_types,
    energy_models,
    equation_manager as equation_manager_module,
    constants,
    chemistry,
)

T_tr_init = 15000.0  # K
T_V_init = 300.0  # K

Y_N2_init = 0.767  # initial mole fraction of N2
Y_O2_init = 0.233  # initial mole fraction of O2
Y_N_init = 0.0  # initial mole fraction of N
Y_O_init = 0.0  # initial mole fraction of O
Y_NO_init = 0.0  # initial mole fraction of NO

p_init = 20.42 * constants.ATM_TO_PA  # Pa

species_names = ["N2", "N", "O2", "O", "NO"]


general_species_data_path = (
    "/home/hhoechter/tum/jaxfluids_internship/data/" "air_5_gnoffo.json"
)
gnoffo_equilibrium_enthalpy_data_path = (
    "/home/hhoechter/tum/jaxfluids_internship/data/"
    "air_5_gnoffo_equilibrium_enthalpy.json"
)
bird_equilibrium_enthalpy_data_path = (
    "/home/hhoechter/tum/jaxfluids_internship/data/" "air_5_bird_energy.json"
)
reaction_data_path = "/home/hhoechter/tum/jaxfluids_internship/data/park_1990_reactions.json"
# reaction_data_path = "/home/hhoechter/tum/jaxfluids_internship/data/scanlon_table2_reactions.json"
# reaction_data_path = "/home/hhoechter/tum/jaxfluids_internship/data/casseau_qk_reactions.json"

dt = 1e-10  # s
t_final = 1e-7  # s
save_interval = 1  # every x steps
dx = 1e-4  # m  # TODO: show that this has no impact on the results in 0D

# ------------------------------ Simulation setup ------------------------------

energy_model_config_gnoffo = energy_models.EnergyModelConfig(
    model="gnoffo",
    include_electronic=True,
    data_path=gnoffo_equilibrium_enthalpy_data_path,
)

energy_model_config_bird = energy_models.EnergyModelConfig(
    model="bird",
    include_electronic=True,
    data_path=bird_equilibrium_enthalpy_data_path,
)

species = chemistry_utils.load_species_table(
    general_data_path=general_species_data_path,
    species_names=species_names,
    energy_model_config=energy_model_config_bird,
)

# chemistry_model_config = chemistry.ChemistryModelConfig(model="cvdv_qp")
chemistry_model_config = chemistry.ChemistryModelConfig(model="park", park_vibrational_source="preferential_constant", park_alpha=0.5, qp_constant=0.3)
# chemistry_model_config = chemistry.ChemistryModelConfig(model="park", park_vibrational_source="nonpreferential", park_alpha=0.5, qp_constant=0.3)
reactions = chemistry_utils.load_reactions_from_json(
    json_path=reaction_data_path,
    species_table=species,
    chemistry_model_config=chemistry_model_config,
)

included_reactions, excluded_reactions = chemistry_utils.check_reaction_coverage(
    json_path=reaction_data_path, species_names=species.names
)
print("Included reactions:")
for rxn in included_reactions:
    print(f"  {rxn['equation']}")
print("\nExcluded reactions:")
for rxn in excluded_reactions:
    print(f"  {rxn['equation']} - Missing: {list(rxn['missing_species'])}")

# create NumericsConfig
numerics_config = numerics_types.NumericsConfig(
    dt=dt,
    dx=dx,
    integrator_scheme="forward-euler",
    spatial_scheme="first_order",
    flux_scheme="hllc",
    n_halo_cells=1,
)

# create EquationManager
equation_manager = equation_manager_types.EquationManager(
    species=species,
    collision_integrals=None,
    reactions=reactions,
    numerics_config=numerics_config,
    boundary_condition="periodic",
)

rho_N2 = (
    p_init
    * Y_N2_init
    * species.molar_masses[species.names.index("N2")]
    / (constants.R_universal * T_tr_init)
)
rho_O2 = (
    p_init
    * Y_O2_init
    * species.molar_masses[species.names.index("O2")]
    / (constants.R_universal * T_tr_init)
)
rho_N = (
    p_init
    * Y_N_init
    * species.molar_masses[species.names.index("N")]
    / (constants.R_universal * T_tr_init)
)
rho_O = (
    p_init
    * Y_O_init
    * species.molar_masses[species.names.index("O")]
    / (constants.R_universal * T_tr_init)
)
rho_NO = (
    p_init  
    * Y_NO_init
    * species.molar_masses[species.names.index("NO")]
    / (constants.R_universal * T_tr_init)
)


# initial condition
U_init = equation_manager_utils.compute_U_from_primitives(
    Y_s=jnp.array([Y_N2_init, Y_N_init, Y_O2_init, Y_O_init, Y_NO_init]),
    rho=jnp.array([(rho_N2 + rho_O2 + rho_N + rho_O + rho_NO)]),
    u=jnp.array([0.0]),
    T_tr=jnp.array([T_tr_init]),
    T_V=jnp.array([T_V_init]),
    equation_manager=equation_manager,
)

# run simulation
U_field, t = equation_manager_module.run_scan(
    U_init=U_init,
    equation_manager=equation_manager,
    t_final=t_final,
    save_interval=save_interval,
)
print("Simulation completed.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Included reactions:
  O2 + N -> O + O + N
  O2 + NO -> O + O + NO
  O2 + N2 -> O + O + N2
  O2 + O2 -> O + O + O2
  O2 + O -> O + O + O
  N2 + O -> N + N + O
  N2 + O2 -> N + N + O2
  N2 + NO -> N + N + NO
  N2 + N2 -> N + N + N2
  N2 + N -> N + N + N
  NO + N2 -> N + O + N2
  NO + O2 -> N + O + O2
  NO + NO -> N + O + NO
  NO + O -> N + O + O
  NO + N -> N + O + N
  NO + O -> O2 + N
  N2 + O -> NO + N
  O2 + N -> NO + O
  NO + N -> N2 + O

Excluded reactions:
Simulation completed.


In [46]:
from plotly import graph_objects as go
import pandas as pd

# JIT-compiled version of extract_primitives_from_U (needs to be executed once per kernel uptime)
extract_primitives_from_U_jitted = jax.jit(
    equation_manager_utils.extract_primitives_from_U
)

# extract primitives and plot
Y_s, rho, T, T_V, p = jax.vmap(
    extract_primitives_from_U_jitted,
    in_axes=(0, None),
)(U_field, equation_manager)


fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=t,
        y=T_V[:, 0],
        mode="lines",
        name="T_V",
        line=dict(shape="spline", smoothing=1.0),
    )
)
fig.add_trace(
    go.Scatter(
        x=t, y=T[:, 0], mode="lines", name="T", line=dict(shape="spline", smoothing=1.0)
    )
)

fig.add_hline(y=7623.3, name="T_eq Casseau", line_dash="dash", line_color="black")

# Load and plot reference data from CSV
csv_path = "/home/hhoechter/tum/jaxfluids_internship/experiments/heatbath_0d/williams_figure_1_air.csv"
df = pd.read_csv(
    csv_path, skiprows=1
)  # Skip the first header row, use X,Y,X,Y... as columns

# Read first row to get the dataset names
with open(csv_path, "r") as f:
    header_line = f.readline().strip()
dataset_names = [name for name in header_line.split(",") if name]

# Group datasets by prefix (e.g., "modified", "hy2foam_default", "monaco")
prefixes = []
for name in dataset_names:
    # Extract prefix by removing _t_v or _t_tr suffix
    if "_t_v" in name:
        prefix = name.replace("_t_v", "")
    elif "_t_tr" in name:
        prefix = name.replace("_t_tr", "")
    else:
        prefix = name
    if prefix not in prefixes:
        prefixes.append(prefix)

# Define colors for each prefix
colors = ["green", "red", "purple", "orange", "brown", "pink"]
prefix_colors = {prefix: colors[i % len(colors)] for i, prefix in enumerate(prefixes)}

# Plot each dataset (columns come in pairs: X, Y for each dataset)
for i, name in enumerate(dataset_names):
    x_col = i * 2  # X column index
    y_col = i * 2 + 1  # Y column index

    if x_col >= len(df.columns) or y_col >= len(df.columns):
        continue

    x_data = pd.to_numeric(df.iloc[:, x_col], errors="coerce").dropna().values
    y_data = pd.to_numeric(df.iloc[:, y_col], errors="coerce").dropna().values

    # Use minimum length in case of mismatched data
    min_len = min(len(x_data), len(y_data))
    x_data = x_data[:min_len]
    y_data = y_data[:min_len]

    # Find prefix for this dataset
    if "_t_v" in name:
        prefix = name.replace("_t_v", "")
    elif "_t_tr" in name:
        prefix = name.replace("_t_tr", "")
    else:
        prefix = name

    fig.add_trace(
        go.Scatter(
            x=x_data,
            y=y_data * 1000,  # Scale Y values (they appear to be in units of 1000 K)
            mode="markers+lines",
            line=dict(dash="dot", smoothing=1.0, shape="spline"),
            name=name,
            marker=dict(color=prefix_colors.get(prefix, "gray")),
        )
    )

fig.update_xaxes(type="log", exponentformat="power", showexponent="all", showgrid=True)
fig.update_yaxes(range=[0, 16000], showgrid=True)
fig.update_layout(
    template="simple_white",
    title="Energy Relaxation Correlation with Casseau for nonreacting N2",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (K)",
    legend=dict(
        x=0.95,
        y=0.05,
        xanchor="right",
        yanchor="top",
        borderwidth=1,
        bordercolor="black",
    ),
    showlegend=True,
    width=800,
    height=600,
)
fig.show()

# plot normalized species number densities
M = jnp.sum(Y_s * species.molar_masses[None, None, :], axis=2)
n = Y_s[:, 0, :] * rho / M * constants.N_A  # 1/m3
n_tot_init = p_init / (constants.R_universal * T_tr_init) * constants.N_A

palette = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b"]

fig = go.Figure()
for i, name in enumerate(species.names):
    fig.add_trace(
        go.Scatter(
            x=t,
            y=n[:, i] / n_tot_init,
            mode="lines",
            name=f"{name} mole fraction",
            line=dict(shape="spline", smoothing=1.0, color=palette[i % len(palette)]),
        )
    )

fig.update_xaxes(type="log", exponentformat="power", showexponent="all", showgrid=True)
fig.update_layout(
    template="simple_white",
    title="Species number densities over time",
    xaxis_title="Time (s)",
    yaxis_title="normalized number density",
    legend=dict(
        x=0.95,
        y=0.05,
        xanchor="right",
        yanchor="bottom",
        borderwidth=1,
        bordercolor="black",
    ),
    showlegend=True,
    width=800,
    height=600,
)
fig.update_xaxes(range=[-10, -6])
fig.show()


In [40]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

files = {
    "casseau_qk": "/home/hhoechter/tum/jaxfluids_internship/data/casseau_qk_reactions.json",
    "park": "/home/hhoechter/tum/jaxfluids_internship/data/park_reactions.json",
    "scanlon": "/home/hhoechter/tum/jaxfluids_internship/data/scanlon_table2_reactions.json",
    "park1990": "/home/hhoechter/tum/jaxfluids_internship/data/park_1990_reactions.json",
}

def load_rates(path):
    data = json.loads(Path(path).read_text())
    rates = {}
    for rxn in data.get("reactions", []):
        eq = rxn.get("equation", "")
        rates[eq] = {
            "C_f": rxn.get("C_f"),
            "n_f": rxn.get("n_f"),
            "E_f_over_k": rxn.get("E_f_over_k"),
        }
    return rates

rates_by_source = {name: load_rates(path) for name, path in files.items()}

# Union of reactions (preserve order from first source)
order = []
seen = set()
for src in files.keys():
    for eq in rates_by_source[src]:
        if eq not in seen:
            order.append(eq)
            seen.add(eq)

def build_df(param):
    rows = []
    for eq in order:
        row = {src: rates_by_source[src].get(eq, {}).get(param, np.nan) for src in files.keys()}
        rows.append(row)
    return pd.DataFrame(rows, index=order)

df_Cf = build_df("C_f")
df_nf = build_df("n_f")
df_Ek = build_df("E_f_over_k")

def luminance(rgb):
    r, g, b = rgb
    return 0.2126*r + 0.7152*g + 0.0722*b

def global_colors_and_font(df, log_scale=False):
    vals = df.values.astype(float)
    if log_scale:
        vals = np.where(vals > 0, np.log10(vals), np.nan)

    mask = np.isfinite(vals)
    if mask.sum() == 0:
        return [["#f0f0f0"] * df.shape[1] for _ in range(df.shape[0])], \
               [["black"] * df.shape[1] for _ in range(df.shape[0])]

    vmin, vmax = np.nanmin(vals), np.nanmax(vals)
    if np.isclose(vmin, vmax):
        norm = np.full_like(vals, 0.5, dtype=float)
    else:
        norm = (vals - vmin) / (vmax - vmin)

    colors = []
    fonts = []
    for i in range(df.shape[0]):
        row_colors = []
        row_fonts = []
        for j in range(df.shape[1]):
            if not np.isfinite(vals[i, j]):
                row_colors.append("#f0f0f0")
                row_fonts.append("black")
                continue
            r = int(68 + 187*norm[i, j])
            g = int(1 + 180*norm[i, j])
            b = int(84 + 20*(1 - norm[i, j]))
            row_colors.append(f"rgb({r},{g},{b})")
            row_fonts.append("white" if luminance((r, g, b)) < 140 else "black")
        colors.append(row_colors)
        fonts.append(row_fonts)
    return colors, fonts

def format_vals(df):
    return df.applymap(lambda x: "" if not np.isfinite(x) else f"{x:.4g}")

def make_table(df, title, log_scale):
    cell_text = [df.index.tolist()] + [format_vals(df)[col].tolist() for col in df.columns]
    cell_colors = [["#ffffff"] * len(df.index)]
    cell_fonts = [["black"] * len(df.index)]

    colors, fonts = global_colors_and_font(df, log_scale=log_scale)
    for col_i in range(len(df.columns)):
        cell_colors.append([colors[row_i][col_i] for row_i in range(len(df.index))])
        cell_fonts.append([fonts[row_i][col_i] for row_i in range(len(df.index))])

    return go.Table(
        header=dict(
            values=["Reaction"] + list(df.columns),
            fill_color="#dddddd",
            align="left",
            font=dict(size=12, color="black"),
        ),
        cells=dict(
            values=cell_text,
            fill_color=cell_colors,
            font=dict(size=10, color=cell_fonts),
            align="left",
        ),
        name=title,
    )

fig = make_subplots(
    rows=1, cols=3,
    specs=[[{"type": "table"}, {"type": "table"}, {"type": "table"}]],
    subplot_titles=["C_f (global color)", "n_f (global color)", "E_f_over_k (global color)"],
)

fig.add_trace(make_table(df_Cf, "C_f", log_scale=True), row=1, col=1)
fig.add_trace(make_table(df_nf, "n_f", log_scale=False), row=1, col=2)
fig.add_trace(make_table(df_Ek, "E_f_over_k", log_scale=True), row=1, col=3)

fig.update_layout(
    height=1200,
    width=1400,
    title_text="Arrhenius Coefficients (global normalization across reactions)",
)
fig.show()


/tmp/ipykernel_90347/3649336585.py:89: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



## VT Relaxation with Dissociation in Pure N2

In [13]:
%load_ext autoreload
%autoreload 2

import jax  
import jax.numpy as jnp

from compressible_1d import (
    equation_manager_types,
    equation_manager_utils,
    chemistry_types,
    chemistry_utils,
    numerics_types,
    energy_models,
    constants,
    equation_manager as equation_manager_module,
    chemistry,
)

T_tr_init = 20000.0  # K
T_V_init = 300.0  # K

Y_N2_init = 1.0  # initial mole fraction of N2
Y_N_init = 0.0  # initial mole fraction of N

p_init = 27.25 * constants.ATM_TO_PA  # Pa

species_names = ["N2", "N"]

general_species_data_path = (
    "/home/hhoechter/tum/jaxfluids_internship/data/" "air_5_gnoffo.json"
)
gnoffo_equilibrium_enthalpy_data_path = (
    "/home/hhoechter/tum/jaxfluids_internship/data/"
    "air_5_gnoffo_equilibrium_enthalpy.json"
)
bird_equilibrium_enthalpy_data_path = (
    "/home/hhoechter/tum/jaxfluids_internship/data/" "air_5_bird_energy.json"
)
# reaction_data_path = "/home/hhoechter/tum/jaxfluids_internship/data/park_reactions.json"
reaction_data_path = "/home/hhoechter/tum/jaxfluids_internship/data/park_1990_reactions.json"


dt = 1e-10  # s
t_final = 1e-7  # s
save_interval = 1  # every x steps
dx = 1e-4  # m  # TODO: show that this has no impact on the results in 0D

# ------------------------------ Simulation setup ------------------------------

energy_model_config_gnoffo = energy_models.EnergyModelConfig(
    model="gnoffo",
    include_electronic=True,
    data_path=gnoffo_equilibrium_enthalpy_data_path,
)

energy_model_config_bird = energy_models.EnergyModelConfig(
    model="bird",
    include_electronic=True,
    data_path=bird_equilibrium_enthalpy_data_path,
)

species = chemistry_utils.load_species_table(
    general_data_path=general_species_data_path,
    species_names=species_names,
    energy_model_config=energy_model_config_bird,
)

# chemistry_model_config = reaction_rates.ChemistryModelConfig(model="cvdv_qp")
# chemistry_model_config = chemistry.ChemistryModelConfig(model="park", park_vibrational_source="preferential_constant", park_alpha=0.5, qp_constant=0.3)
chemistry_model_config = chemistry.ChemistryModelConfig(model="park", park_vibrational_source="nonpreferential", park_alpha=0.5)
reactions = chemistry_utils.load_reactions_from_json(
    json_path=reaction_data_path,
    species_table=species,
    chemistry_model_config=chemistry_model_config,
)

included_reactions, excluded_reactions = chemistry_utils.check_reaction_coverage(
    json_path=reaction_data_path, species_names=species.names
)
print("Included reactions:")
for rxn in included_reactions:
    print(f"  {rxn['equation']}")
print("\nExcluded reactions:")
for rxn in excluded_reactions:
    print(f"  {rxn['equation']} - Missing: {list(rxn['missing_species'])}")

# create NumericsConfig
numerics_config = numerics_types.NumericsConfig(
    dt=dt,
    dx=dx,
    integrator_scheme="forward-euler",
    spatial_scheme="first_order",
    flux_scheme="hllc",
    n_halo_cells=1,
)

# create EquationManager
equation_manager = equation_manager_types.EquationManager(
    species=species,
    collision_integrals=None,
    reactions=reactions,
    numerics_config=numerics_config,
    boundary_condition="periodic",
)

rho_N2 = (
    p_init
    * Y_N2_init
    * species.molar_masses[species.names.index("N2")]
    / (constants.R_universal * T_tr_init)
)

rho_N = (
    p_init
    * Y_N_init
    * species.molar_masses[species.names.index("N")]
    / (constants.R_universal * T_tr_init)
)

# initial condition
U_init = equation_manager_utils.compute_U_from_primitives(
    Y_s=jnp.array([Y_N2_init, Y_N_init]),
    rho=jnp.array([(rho_N2 + rho_N)]),
    u=jnp.array([0.0]),
    T_tr=jnp.array([T_tr_init]),
    T_V=jnp.array([T_V_init]),
    equation_manager=equation_manager,
)

# run simulation
U_field, t = equation_manager_module.run_scan(
    U_init=U_init,
    equation_manager=equation_manager,
    t_final=t_final,
    save_interval=save_interval,
)
print("Simulation completed.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Included reactions:
  N2 + N2 -> N + N + N2
  N2 + N -> N + N + N

Excluded reactions:
  O2 + N -> O + O + N - Missing: ['O2', 'O']
  O2 + NO -> O + O + NO - Missing: ['NO', 'O2', 'O']
  O2 + N2 -> O + O + N2 - Missing: ['O2', 'O']
  O2 + O2 -> O + O + O2 - Missing: ['O2', 'O']
  O2 + O -> O + O + O - Missing: ['O2', 'O']
  N2 + O -> N + N + O - Missing: ['O']
  N2 + O2 -> N + N + O2 - Missing: ['O2']
  N2 + NO -> N + N + NO - Missing: ['NO']
  NO + N2 -> N + O + N2 - Missing: ['NO', 'O']
  NO + O2 -> N + O + O2 - Missing: ['NO', 'O2', 'O']
  NO + NO -> N + O + NO - Missing: ['NO', 'O']
  NO + O -> N + O + O - Missing: ['NO', 'O']
  NO + N -> N + O + N - Missing: ['NO', 'O']
  NO + O -> O2 + N - Missing: ['NO', 'O2', 'O']
  N2 + O -> NO + N - Missing: ['NO', 'O']
  O2 + N -> NO + O - Missing: ['NO', 'O2', 'O']
  NO + N -> N2 + O - Missing: ['NO', 'O']
Simulation completed.


In [14]:
from plotly import graph_objects as go
import pandas as pd

# JIT-compiled version of extract_primitives_from_U (needs to be executed once per kernel uptime)
extract_primitives_from_U_jitted = jax.jit(
    equation_manager_utils.extract_primitives_from_U
)

# extract primitives and plot
Y_s, rho, T, T_V, p = jax.vmap(
    extract_primitives_from_U_jitted,
    in_axes=(0, None),
)(U_field, equation_manager)


fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=t,
        y=T_V[:, 0],
        mode="lines",
        name="T_V",
        line=dict(shape="spline", smoothing=1.0),
    )
)
fig.add_trace(
    go.Scatter(
        x=t, y=T[:, 0], mode="lines", name="T", line=dict(shape="spline", smoothing=1.0)
    )
)

fig.add_hline(y=7623.3, name="T_eq Casseau", line_dash="dash", line_color="black")

# Load and plot reference data from CSV
csv_path = "/home/hhoechter/tum/jaxfluids_internship/experiments/heatbath_0d/williams_figure_1_n2.csv"
df = pd.read_csv(
    csv_path, skiprows=1
)  # Skip the first header row, use X,Y,X,Y... as columns

# Read first row to get the dataset names
with open(csv_path, "r") as f:
    header_line = f.readline().strip()
dataset_names = [name for name in header_line.split(",") if name]

# Group datasets by prefix (e.g., "modified", "hy2foam_default", "monaco")
prefixes = []
for name in dataset_names:
    # Extract prefix by removing _t_v or _t_tr suffix
    if "_t_v" in name:
        prefix = name.replace("_t_v", "")
    elif "_t_tr" in name:
        prefix = name.replace("_t_tr", "")
    else:
        prefix = name
    if prefix not in prefixes:
        prefixes.append(prefix)

# Define colors for each prefix
colors = ["green", "red", "purple", "orange", "brown", "pink"]
prefix_colors = {prefix: colors[i % len(colors)] for i, prefix in enumerate(prefixes)}

# Plot each dataset (columns come in pairs: X, Y for each dataset)
for i, name in enumerate(dataset_names):
    x_col = i * 2  # X column index
    y_col = i * 2 + 1  # Y column index

    if x_col >= len(df.columns) or y_col >= len(df.columns):
        continue

    x_data = pd.to_numeric(df.iloc[:, x_col], errors="coerce").dropna().values
    y_data = pd.to_numeric(df.iloc[:, y_col], errors="coerce").dropna().values

    # Use minimum length in case of mismatched data
    min_len = min(len(x_data), len(y_data))
    x_data = x_data[:min_len]
    y_data = y_data[:min_len]

    # Find prefix for this dataset
    if "_t_v" in name:
        prefix = name.replace("_t_v", "")
    elif "_t_tr" in name:
        prefix = name.replace("_t_tr", "")
    else:
        prefix = name

    fig.add_trace(
        go.Scatter(
            x=x_data,
            y=y_data * 1000,  # Scale Y values (they appear to be in units of 1000 K)
            mode="markers+lines",
            line=dict(dash="dot", smoothing=1.0, shape="spline"),
            name=name,
            marker=dict(color=prefix_colors.get(prefix, "gray")),
        )
    )

fig.update_yaxes(range=[0, 20500], showgrid=True)
fig.update_layout(
    template="simple_white",
    title="Energy Relaxation Correlation with Casseau for nonreacting N2",
    xaxis_title="Time (s)",
    yaxis_title="Temperature (K)",
    legend=dict(
        x=0.95,
        y=0.05,
        xanchor="right",
        yanchor="bottom",
        borderwidth=1,
        bordercolor="black",
    ),
    showlegend=True,
    width=800,
    height=600,
)
fig.show()
fig.update_xaxes(type="log", exponentformat="power", showexponent="all", showgrid=True)
fig.update_xaxes(range=[-10, -6])
fig.show()

# plot normalized species number densities
M = jnp.sum(Y_s * species.molar_masses[None, None, :], axis=2)
n = Y_s[:, 0, :] * rho / M * constants.N_A  # 1/m3
n_tot_init = p_init / (constants.R_universal * T_tr_init) * constants.N_A

palette = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b"]

fig = go.Figure()
for i, name in enumerate(species.names):
    fig.add_trace(
        go.Scatter(
            x=t,
            y=n[:, i] / n_tot_init,
            mode="lines",
            name=f"{name} mole fraction",
            line=dict(shape="spline", smoothing=1.0, color=palette[i % len(palette)]),
        )
    )

fig.update_xaxes(type="log", exponentformat="power", showexponent="all", showgrid=True)
fig.update_layout(
    template="simple_white",
    title="Species number densities over time",
    xaxis_title="Time (s)",
    yaxis_title="normalized number density",
    legend=dict(
        x=0.95,
        y=0.05,
        xanchor="right",
        yanchor="bottom",
        borderwidth=1,
        bordercolor="black",
    ),
    showlegend=True,
    width=800,
    height=600,
)
fig.update_xaxes(range=[-10, -6])
fig.show()




In [2]:
import json
import numpy as np
import plotly.graph_objects as go

with open("../../data/park_reactions.json") as f:
    data = json.load(f)

n2_reaction = next(r for r in data["reactions"] if r["table_index"] == 9)
coeffs_list = n2_reaction["equilibrium_coeffs_casseau"]

T = np.linspace(300, 30000, 2000)
x = 1e4 / T

fig = go.Figure()

for entry in coeffs_list:
    n_ref = entry["n_ref_cm3"]
    A1, A2, A3, A4, A5 = entry["A0"], entry["A1"], entry["A2"], entry["A3"], entry["A4"]

    exponent = A1 * (T / 1e4) + A2 + A3 * np.log(x) + A4 * x + A5 * x**2
    K_eq = np.exp(exponent)

    fig.add_trace(go.Scatter(x=T, y=K_eq, mode="lines", name=f"n_ref = {n_ref:.0e} cm⁻³"))


# Park Table III equilibrium constant fit for N2 dissociation
# Kc = exp(B1 + B2*ln(Z) + B3*Z + B4*Z^2 + B5*Z^3), Z=10000/T
B1 = 1.858
B2 = -1.325
B3 = -9.856
B4 = -0.174
B5 = 0.008

Z = 1.0e4 / T
Kc_park = np.exp(B1 + B2 * np.log(Z) + B3 * Z + B4 * Z**2 + B5 * Z**3)

fig.add_trace(
    go.Scatter(
        x=T,
        y=Kc_park,
        mode="lines",
        name="Park Kc (N2 dissociation)",
        line=dict(color="black", dash="dash"),
    )
)
fig.update_layout(
    title="N2 Dissociation Equilibrium Constant (Park/Casseau)",
    xaxis_title="Temperature [K]",
    yaxis_title="K_eq (Casseau, dimensionless)",
    yaxis_type="log",
    template="plotly_white",
)
fig.show()



